In [4]:
import datetime
import os
import numpy as np
import json
import calendar
from datetime import date
import httplib2
from apiclient.discovery import build
from oauth2client.client import flow_from_clientsecrets
from oauth2client.file import Storage
from oauth2client import tools
import argparse
 
CLIENT_SECRETS = 'client_secrets.json'
 
# The Flow object to be used if we need to authenticate.
FLOW = flow_from_clientsecrets(
    CLIENT_SECRETS,
    scope='https://www.googleapis.com/auth/analytics.readonly',
    message='%s is missing' % CLIENT_SECRETS
    )
 
# A file to store the access token
TOKEN_FILE_NAME = 'credentials.dat'
 
def prepare_credentials():
    parser = argparse.ArgumentParser(parents=[tools.argparser])
    flags = parser.parse_args()
    # Retrieve existing credendials
    storage = Storage(TOKEN_FILE_NAME)
    credentials = storage.get()
    # If no credentials exist, we create new ones
    if credentials is None or credentials.invalid:
        credentials = tools.run_flow(FLOW, storage, flags)
    return credentials
 
def initialize_service():
    # Creates an http object and authorize it using
    # the function prepare_creadentials()
    http = httplib2.Http()
    credentials = prepare_credentials()
    http = credentials.authorize(http)
    # Build the Analytics Service Object with the authorized http object
    return build('analytics', 'v3', http=http)

def get_accounts_ids(service):
    accounts = service.management().accounts().list().execute()
    ids = []
    if accounts.get('items'): # Thsi can be replaced with a cleaner implementation
        for account in accounts['items']:
            ids.append(account['id'])
    return ids

def get_source_group(service, views_id, start_date, end_date):
    ids = "ga:" + views_id
    metrics = "ga:transactionRevenue, ga:transactionShipping,ga:sessions,ga:sessions"
    dimensions = "ga:transactionId,ga:dimension2"
    data = service.data().ga().get(
        ids=ids, start_date=start_date, end_date=end_date, metrics=metrics,
        dimensions=dimensions).execute()
    try:
        return data['rows']
    except:
        return False
    

#read the session info from GA

def get_chcost():
    sessions_dic={}
    service = initialize_service() 
    views_id = '64933282'#This can be found in your web sites's Google Analytics information page
    results=[]
    for day in ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21']:
        start_date = '2016' + '-' + '01' +'-' + day
        data = get_source_group(service, views_id , start_date, start_date)
        for things in data:
            results.append(things)
    return results
    
data=get_chcost()

SystemExit: client_secrets.json is missing

To exit: use 'exit', 'quit', or Ctrl-D.
